In [7]:
import csv
import re

def load_dataset(filename):
    dataset = {}
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            number = int(row[0])
            meru_number = row[1]
            grammar = row[2]
            morphemes = re.findall(r'\b\w+\b', meru_number)
            dataset[number] = (meru_number, grammar, morphemes)
    return dataset

def analyze_meru_number(input_number, dataset):
    if input_number in dataset:
        # Number is in the dataset
        return dataset[input_number]

    if input_number < 1000:
        hundreds = input_number // 100
        remainder = input_number % 100
        if remainder == 0:
            meru_number = f"{dataset[hundreds][0]} {dataset[100][0]}"
            grammar = f"{dataset[hundreds][1]} {dataset[100][1]}"
        else:
            tens = remainder // 10
            ones = remainder % 10
            if tens == 0:
                meru_number = f"{dataset[hundreds][0]} {dataset[100][0]} {dataset[ones][0]}"
                grammar = f"{dataset[hundreds][1]} {dataset[100][1]} {dataset[ones][1]}"
            else:
                if tens * 10 in dataset:
                    meru_number = f"{dataset[hundreds][0]} {dataset[100][0]} {dataset[tens * 10][0]} na {dataset[ones][0]}"
                    grammar = f"{dataset[hundreds][1]} {dataset[100][1]} {dataset[tens * 10][1]} na {dataset[ones][1]}"
                else:
                    meru_number = f"{dataset[hundreds][0]} {dataset[100][0]} {dataset[ones][0]}"
                    grammar = f"{dataset[hundreds][1]} {dataset[100][1]} {dataset[ones][1]}"
        return meru_number, grammar, dataset[hundreds][2] + dataset[100][2] + dataset[tens * 10][2] + dataset[ones][2]

    if input_number >= 1000:
        thousands = input_number // 1000
        remainder = input_number % 1000
        meru_number_parts = []

        if thousands == 1:
            meru_number_parts.append(dataset[1000][0])
            meru_number_parts.append(dataset[remainder][0])
            grammar_parts = [dataset[1000][1], dataset[remainder][1]]
            morphemes_parts = dataset[1000][2] + dataset[remainder][2]
        else:
            meru_number_parts.append(dataset[thousands][0])
            meru_number_parts.append(dataset[1000][0])
            if remainder > 0:
                hundreds = remainder // 100
                meru_number_parts.append(dataset[hundreds][0])
                remainder = remainder % 100
                if remainder > 0:
                    tens = remainder // 10
                    ones = remainder % 10
                    if tens * 10 in dataset:
                        meru_number_parts.append(f"{dataset[tens * 10][0]} na {dataset[ones][0]}")
                        grammar_parts = [
                            dataset[thousands][1],
                            dataset[1000][1],
                            dataset[hundreds][1],
                            dataset[tens * 10][1],
                            dataset[ones][1]
                        ]
                        morphemes_parts = (
                            dataset[thousands][2]
                            + dataset[1000][2]
                            + dataset[hundreds][2]
                            + dataset[tens * 10][2]
                            + dataset[ones][2]
                        )
                    else:
                        meru_number_parts.append(dataset[ones][0])
                        grammar_parts = [
                            dataset[thousands][1],
                            dataset[1000][1],
                            dataset[hundreds][1],
                            dataset[ones][1]
                        ]
                        morphemes_parts = (
                            dataset[thousands][2]
                            + dataset[1000][2]
                            + dataset[hundreds][2]
                            + dataset[ones][2]
                        )
                else:
                    grammar_parts = [dataset[thousands][1], dataset[1000][1], dataset[hundreds][1]]
                    morphemes_parts = dataset[thousands][2] + dataset[1000][2] + dataset[hundreds][2]
            else:
                grammar_parts = [dataset[thousands][1], dataset[1000][1]]
                morphemes_parts = dataset[thousands][2] + dataset[1000][2]

        meru_number = " ".join(meru_number_parts)
        grammar = " ".join(grammar_parts)
        return meru_number, grammar, morphemes_parts

def run_meru_number_analyzer():
    dataset = load_dataset('MeruNumbers.csv')
    while True:
        print("Options:")
        print("1. Number to Meru conversion")
        print("2. Meru to Number conversion")
        print("3. Quit")
        choice = input("Enter your choice (1-3): ")

        if choice == '1':
            # Number to Meru conversion
            input_number = int(input("Enter a number: "))
            meru_number, grammar, morphemes = analyze_meru_number(input_number, dataset)
            print(f"The Meru number for {input_number} is: {meru_number} ({grammar})")
            print(f"Morphemes: {', '.join(morphemes)}")
            print()
        elif choice == '2':
            # Meru to Number conversion
            input_meru = input("Enter a Meru number: ")
            found_number = False
            for number, (meru, _, _) in dataset.items():
                if input_meru.strip().lower() == meru.strip().lower():
                    print(f"The number equivalent of {input_meru} is: {number}")
                    found_number = True
                    break
            if not found_number:
                print("Meru number not found in the dataset.")
            print()
        elif choice == '3':
            # Quit the program
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please enter a valid option (1-3).")
            print()

run_meru_number_analyzer()


Options:
1. Number to Meru conversion
2. Meru to Number conversion
3. Quit


UnboundLocalError: cannot access local variable 'tens' where it is not associated with a value